In [1]:
import torch
from torch.autograd import Variable, grad

In [20]:
x = Variable(torch.rand(2, 2) + torch.eye(2), requires_grad=True)

# inverse, Cholesky, QR
for f in [torch.inverse, torch.potrf, torch.geqrf]:
    y = x.potrf()
    y.sum().backward()
    x.grad

In [30]:
x = Variable(torch.randn(3, 4), requires_grad=True)
y = torch.nn.functional.relu(x)
dx = grad(y, x, torch.ones_like(y))
ddx = grad(dx, x, torch.ones_like(y))
ddx
# error no grad_fn: dddx = grad(ddx, x, torch.ones_like(y))

(Variable containing:
  0  0  0  0
  0  0  0  0
  0  0  0  0
 [torch.FloatTensor of size 3x4],)

In [71]:
ddx.grad_fn.next_functions

((<AddBackward_Deprecated at 0x7efdada014e0>, 0),
 (<torch.autograd.function.SetItemBackward at 0x7efdad8e9ca8>, 0))

In [53]:
def f(x):
    return (x[0] - 1)**2 + 10 * (x[0]**2 - x[1])**2

x = Variable(torch.randn(2), requires_grad=True)
y = f(x)
# scalar variable has implicit output_grad = 1.0
# create_graph make dx.volatile=False
dx = grad(y, x, create_graph=True)[0]
# non scalar variable requries explict output_grad
ddx = grad(dx, x, torch.ones_like(dx))[0]  # []

# grad frees intermediate grads by default
ddx0 = grad(dx[0], x, retain_graph=True)[0]  # [df/dx0dx0, df/dx0dx1]
ddx1 = grad(dx[1], x)[0] # [df/dx1dx0, df/dx1dx1]
hesse = torch.stack((ddx0, ddx1))
print(ddx, ddx0 + ddx1)

Variable containing:
 19.0546
  8.1308
[torch.FloatTensor of size 2]
 Variable containing:
 19.0546
  8.1308
[torch.FloatTensor of size 2]



In [54]:
hesse

Variable containing:
 30.9238 -11.8692
-11.8692  20.0000
[torch.FloatTensor of size 2x2]

In [56]:
ddx1

Variable containing:
-11.8692
 20.0000
[torch.FloatTensor of size 2]